In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

C:\Users\Admin\.conda\envs\CV\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Admin\.conda\envs\CV\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Admin\.conda\envs\CV\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Admin\.conda\envs\CV\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

1.14.0


Using TensorFlow backend.


In [2]:
training_set = pd.read_csv("train.csv")

training_set.columns

training_labels_1 = training_set['emergency-or-not']

training_imgs = ["{}".format(x) for x in list(training_set.image_names)]

training_set = pd.DataFrame({'Images': training_imgs,'emergency-or-not': training_labels_1})

training_set['emergency-or-not'] = training_set['emergency-or-not'].astype(str)



In [3]:
print(training_set.head())



     Images emergency-or-not
0  1503.jpg              not
1  1420.jpg              not
2  1764.jpg              not
3  1356.jpg              not
4  1117.jpg              not


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dataGen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

train_generator = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set,
                                        directory="",x_col="Images",
                                        y_col="emergency-or-not",
                                        class_mode="categorical",
                                        #target_size=(128,128),
                                        target_size=(224,224),
                                        batch_size=32)




Found 1646 validated image filenames belonging to 2 classes.


In [5]:
classifier = Sequential()



In [6]:
#First Convolutional layer
#classifier.add(Conv2D(filters = 56,kernel_size = (3,3), activation = 'relu', input_shape = (128,128,3)))
classifier.add(Conv2D(filters = 56,kernel_size = (3,3), activation = 'relu', input_shape = (224,224,3)))

In [7]:
classifier.add(MaxPooling2D(pool_size = (2,2)))



In [8]:
#second Convolutional layer
classifier.add(Conv2D(32,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [9]:
#Flattening
classifier.add(Flatten())

In [10]:
#Hidden Layer
classifier.add(Dense(units = 64, activation = 'relu'))

#Output Layer
classifier.add(Dense(units = 2 , activation = 'softmax'))

In [11]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy','accuracy'])

In [12]:
classifier.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 56)      1568      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 56)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 32)      16160     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 93312)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                5972032   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                

In [13]:
classifier.fit_generator(train_generator, epochs = 50, steps_per_epoch = 60 )


Epoch 1/50
60/60 [==============================] - 116s 2s/step - loss: 0.8057 - categorical_accuracy: 0.6751 - accuracy: 0.6751
Epoch 2/50
60/60 [==============================] - 108s 2s/step - loss: 0.4932 - categorical_accuracy: 0.7613 - accuracy: 0.7613
Epoch 3/50
60/60 [==============================] - 110s 2s/step - loss: 0.4767 - categorical_accuracy: 0.7750 - accuracy: 0.7750
Epoch 4/50
60/60 [==============================] - 111s 2s/step - loss: 0.4358 - categorical_accuracy: 0.8065 - accuracy: 0.8065
Epoch 5/50
60/60 [==============================] - 108s 2s/step - loss: 0.4220 - categorical_accuracy: 0.7978 - accuracy: 0.7978
Epoch 6/50
60/60 [==============================] - 105s 2s/step - loss: 0.3932 - categorical_accuracy: 0.8254 - accuracy: 0.8254
Epoch 7/50
60/60 [==============================] - 105s 2s/step - loss: 0.3651 - categorical_accuracy: 0.8344 - accuracy: 0.8344
Epoch 8/50
60/60 [==============================] - 114s 2s/step - loss: 0.3501 - categor

In [14]:
test_set = pd.read_csv("test.csv")
test_imgs = test_set['image_names']
test_set = pd.DataFrame( {'Images': test_imgs })

test_set

,Images
0,1960.jpg
1,668.jpg
2,2082.jpg
3,808.jpg
4,1907.jpg
...,...
701,674.jpg
702,1027.jpg
703,447.jpg
704,2176.jpg


In [15]:
#classes = train_generator.class_indices
classes = {'emergency-or-not': 1, 'not': 0}

print(classes)



{'emergency-or-not': 1, 'not': 0}


In [16]:
inverted_classes = dict(map(reversed, classes.items()))
print(inverted_classes)

{1: 'emergency-or-not', 0: 'not'}


In [17]:
from keras.preprocessing import image

Y_pred = []

for i in range(len(test_set)):
  img = image.load_img(path= test_set.Images[i],target_size=(224,224,3))
  #img = image.load_img(path= test_set.Images[i],target_size=(128,128,3))
  img = image.img_to_array(img)
  test_img = img.reshape((1,224,224,3))
  #test_img = img.reshape((1,128,128,3))
  img_class = classifier.predict(test_img)
  prediction = img_class[0]
  print(test_set.Images[i],",",img_class[0,0])
  #print(test_set.Images[i])
  #print(img_class.shape)
  #print(prediction)
  Y_pred.append(prediction)

1960.jpg , 0.0
668.jpg , 1.0
2082.jpg , 0.0
808.jpg , 1.0
1907.jpg , 0.0
350.jpg , 1.0
1557.jpg , 0.0
111.jpg , 1.0
1952.jpg , 0.0
1466.jpg , 0.0
2071.jpg , 1.0
1891.jpg , 0.0
1900.jpg , 0.0
1927.jpg , 0.0
1414.jpg , 0.0
134.jpg , 1.0
1037.jpg , 0.0
56.jpg , 0.0
208.jpg , 1.0
629.jpg , 1.0
979.jpg , 0.0
2099.jpg , 0.0
332.jpg , 0.0
1814.jpg , 0.0
1094.jpg , 0.0
1886.jpg , 0.0
342.jpg , 0.0
2134.jpg , 0.0
1465.jpg , 0.0
482.jpg , 0.0
229.jpg , 1.0
1078.jpg , 0.0
611.jpg , 0.0
1349.jpg , 0.0
1805.jpg , 1.0
1947.jpg , 0.0
299.jpg , 0.0
463.jpg , 0.0
2154.jpg , 0.0
1244.jpg , 0.0
581.jpg , 1.0
218.jpg , 1.0
1553.jpg , 0.0
1025.jpg , 0.0
1591.jpg , 0.0
1132.jpg , 0.0
2117.jpg , 0.0
1795.jpg , 0.0
2254.jpg , 0.0
1950.jpg , 0.0
507.jpg , 0.0
445.jpg , 0.0
1501.jpg , 0.0
2349.jpg , 0.0
678.jpg , 0.0
1604.jpg , 0.0
792.jpg , 1.0
879.jpg , 0.0
1779.jpg , 0.0
2156.jpg , 0.0
44.jpg , 0.0
686.jpg , 1.0
1311.jpg , 0.0
2192.jpg , 0.0
440.jpg , 1.0
1047.jpg , 0.0
254.jpg , 1.0
479.jpg , 0.0
282.jpg , 

427.jpg , 1.0
105.jpg , 1.0
1301.jpg , 0.0
121.jpg , 1.0
1352.jpg , 0.0
2186.jpg , 0.0
2275.jpg , 0.0
1724.jpg , 0.0
115.jpg , 1.0
1532.jpg , 0.0
2195.jpg , 0.0
685.jpg , 1.0
746.jpg , 0.0
2293.jpg , 0.0
2051.jpg , 0.0
782.jpg , 1.0
1247.jpg , 0.0
742.jpg , 1.0
552.jpg , 0.0
87.jpg , 1.0
1472.jpg , 0.0
2270.jpg , 0.0
1405.jpg , 0.0
1001.jpg , 0.0
1547.jpg , 0.0
1918.jpg , 0.0
543.jpg , 1.0
1535.jpg , 0.0
494.jpg , 0.0
1146.jpg , 0.0
2128.jpg , 0.0
1747.jpg , 0.0
1720.jpg , 0.0
71.jpg , 0.0
63.jpg , 0.0
1220.jpg , 0.0
1255.jpg , 0.0
845.jpg , 0.0
1199.jpg , 0.0
1760.jpg , 0.0
1125.jpg , 1.0
1670.jpg , 0.0
1537.jpg , 0.0
931.jpg , 1.0
1341.jpg , 0.0
205.jpg , 1.0
841.jpg , 1.0
1990.jpg , 0.0
557.jpg , 1.0
377.jpg , 1.0
535.jpg , 1.0
1131.jpg , 0.0
1456.jpg , 0.0
198.jpg , 1.0
561.jpg , 0.0
1061.jpg , 0.0
590.jpg , 1.0
519.jpg , 1.0
308.jpg , 0.0
1876.jpg , 0.0
2229.jpg , 0.0
59.jpg , 0.0
836.jpg , 1.0
2318.jpg , 0.0
207.jpg , 1.0
1949.jpg , 0.0
1351.jpg , 0.0
1488.jpg , 0.0
162.jpg , 1.0

In [18]:
#print(Y_pred)
print(Y_pred[5])

print(inverted_classes)

[1. 0.]
{1: 'emergency-or-not', 0: 'not'}


In [19]:
prediction_classes = [ inverted_classes.get(item[0]) for item in Y_pred ]
prediction_classes


['not',
 'emergency-or-not',
 'not',
 'emergency-or-not',
 'not',
 'emergency-or-not',
 'not',
 'emergency-or-not',
 'not',
 'not',
 'emergency-or-not',
 'not',
 'not',
 'not',
 'not',
 'emergency-or-not',
 'not',
 'not',
 'emergency-or-not',
 'emergency-or-not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'emergency-or-not',
 'not',
 'not',
 'not',
 'emergency-or-not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'emergency-or-not',
 'emergency-or-not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'emergency-or-not',
 'not',
 'not',
 'not',
 'not',
 'emergency-or-not',
 'not',
 'not',
 'emergency-or-not',
 'not',
 'emergency-or-not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'emergency-or-not',
 'not',
 'not',
 'not',
 'not',
 'emergency-or-not',
 'not',
 'emergency-or-not',
 'not',
 'emergency-or-not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'not',
 'emergency-or-not',
 'not',

In [20]:
print(prediction_classes)

pd.DataFrame(predictions).to_csv("predictionss.csv", index = False)



['not', 'emergency-or-not', 'not', 'emergency-or-not', 'not', 'emergency-or-not', 'not', 'emergency-or-not', 'not', 'not', 'emergency-or-not', 'not', 'not', 'not', 'not', 'emergency-or-not', 'not', 'not', 'emergency-or-not', 'emergency-or-not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'emergency-or-not', 'not', 'not', 'not', 'emergency-or-not', 'not', 'not', 'not', 'not', 'not', 'emergency-or-not', 'emergency-or-not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'emergency-or-not', 'not', 'not', 'not', 'not', 'emergency-or-not', 'not', 'not', 'emergency-or-not', 'not', 'emergency-or-not', 'not', 'not', 'not', 'not', 'not', 'emergency-or-not', 'not', 'not', 'not', 'not', 'emergency-or-not', 'not', 'emergency-or-not', 'not', 'emergency-or-not', 'not', 'not', 'not', 'not', 'not', 'not', 'not', 'emergency-or-not', 'not', 'emergency-or-not', 'emergency-or-not', 'emergency-or-not', 'not', 'emergency-or-not', 'e

NameError: name 'predictions' is not defined

In [ ]:
animal = []
breed = []
for i in prediction_classes:
  animal.append(i[0]) # First character = class_name/Animal
  breed.append(i[1:]) # Last 2 characters = breed/Breed

In [ ]:
predictions = {}
predictions['class_name'] = animal
predictions['breed'] = breed

In [ ]:
#Writing to excel
pd.DataFrame(predictions).to_csv("predictionss.csv", index = False)